In [1]:
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
import sys
import random
import pyautogui
from PyQt5 import QtWidgets, uic
import icon_rc_
import webbrowser

In [2]:
# LSTM for international airline passengers problem with memory
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import time
import csv #엑셀파일
import os
import win32com.client 
from openpyxl import Workbook
import xlwt
import math
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import openpyxl 
import matplotlib.pyplot as plt
import keras

Using TensorFlow backend.


In [3]:
basic_ui = uic.loadUiType("UI_design.ui")[0]

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [],[] 
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back):]
        dataX.append(a)
        dataY.append(dataset[i:(i + look_back):])
    
    dataX=np.squeeze(dataX,axis=1) #차원 축소
    dataY=np.squeeze(dataY,axis=1) #차원 축소
    return np.array(dataX), np.array(dataY)

In [ ]:
np.random.seed(7)

data1=pd.read_excel('x_data.xlsx') #x좌표 엑셀 데이터 불러옴 
data2=pd.read_excel('y_data.xlsx') #y좌표 엑셀 데이터 불러옴 
data3=pd.read_excel('speed_data.xlsx') #speed좌표 엑셀 데이터 불러옴 

pddata1=pd.DataFrame(data1) 
pddata1.head()
pddata2=pd.DataFrame(data2) 
pddata2.head()
pddata3=pd.DataFrame(data3) 
pddata3.head()

hap=[] #진짜 합
for j in range(len(pddata1)):
    x_data=np.array(pddata1.loc[j])
    y_data=np.array(pddata2.loc[j])
    s_data=np.array(pddata3.loc[j])

    hap1=[] #[x,y,속력] 데이터를 저장하고 있는 리스트
    
    for i in range(len(s_data)): 
        sum=[] #리스트 하나당 임시로 x,y,속력을 저장할 리스트 (for문 돌릴때마다 초기화)
        sum.append(x_data[i]) #x넣음
        sum.append(y_data[i]) #y넣음
        sum.append(s_data[i]) #속력넣음
        hap1.append(sum) #[x,y,속력] 하나의 리스트를 hap리스트에 넣음  

    j = j + 1
    
    hap.append(hap1)

n1 = np.array(hap)
print(n1.shape)

In [ ]:
num_sample = n1.shape[0] # 257개 Sample 데이터
num_sequence = n1.shape[1] # 99개 시계열 데이터
num_feature = n1.shape[2] #3개 Feature

dataset = n1

In [ ]:
train_size = int(len(dataset) * 0.70) #학습 데이터 70%
test_size = len(dataset) - train_size #테스트 데이터 30%
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [ ]:
look_back = 1 #이전 시간 단계 입력변수
#X는 지금 t 값이고, Y는 그 다음의 t+1 값임 (즉, X=t, Y=t+1)
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
from keras.layers import TimeDistributed
from keras.layers import InputLayer
from keras.layers import Reshape
from keras.layers import *
from keras.models import *
from keras.utils import *

model = Sequential() # Sequeatial Model 
model.add(LSTM(32,return_sequences=True, input_shape=(num_sequence,num_feature)))  
model.add(Dense(3, activation='softmax')) # 3개의 예측
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

history = model.fit(trainX, trainY, epochs=1, batch_size=20, validation_split=0.2, verbose=1)

In [ ]:
print("정확도: ",(model.evaluate(testX, testY)))

In [4]:
#버튼 출력 윈도우창   
class WindowClass(QMainWindow, basic_ui):
    def __init__(self):
        super().__init__()
        self.setupUi(self)

        self.cnt = 0
        
        self.x_data=[]
        self.y_data=[]
        self.speed=[]
        self.n = 0

        self.show()
        
        qr = self.frameGeometry()
        cp = QDesktopWidget().availableGeometry().center()
        qr.moveCenter(cp)
        self.move(qr.topLeft())
        
        self.kwangwoon_btn.clicked.connect(self.ClickMethod)
        self.naver_btn.clicked.connect(self.ClickMethod)
        self.google_btn.clicked.connect(self.ClickMethod)
        self.youtube_btn.clicked.connect(self.ClickMethod)
        self.instagram_btn.clicked.connect(self.ClickMethod)
        self.twitter_btn.clicked.connect(self.ClickMethod)
        self.zoom_btn.clicked.connect(self.ClickMethod)
        self.facebook_btn.clicked.connect(self.ClickMethod)
        self.everytime_btn.clicked.connect(self.ClickMethod)
        self.start_btn.clicked.connect(self.createCsv)
        #self.start_btn.clicked.connect(self.Start) 최종 프로젝트 
        self.show()

    def createCsv(self):

            #엑셀파일에 x좌표값 저장
            sheet1 = ["Sheet 1"]

            wb = xlwt.Workbook()

            for sheet, data in zip(sheet1, [self.x_data]):
                ws = wb.add_sheet(sheet)

                for row, row_value in enumerate(data):
                    for col, col_value in enumerate(row_value):
                        ws.write(row, col, col_value)

            wb.save('x_data.xls')

            #엑셀파일에 y좌표값 저장
            sheet2 = ["Sheet 1"]

            wb = xlwt.Workbook()

            for sheet, data in zip(sheet2, [self.y_data]):
                ws = wb.add_sheet(sheet)

                for row, row_value in enumerate(data):
                    for col, col_value in enumerate(row_value):
                        ws.write(row, col, col_value)

            wb.save('y_data.xls')
            
            #엑셀파일에 속력값 저장
            sheet3 = ["Sheet 1"]

            wb = xlwt.Workbook()

            for sheet, data in zip(sheet3, [self.speed]):
                ws = wb.add_sheet(sheet)

                for row, row_value in enumerate(data):
                    for col, col_value in enumerate(row_value):
                        ws.write(row, col, col_value)

            wb.save('speed.xls')
            

            #x좌표와 y좌표 화면 상 확인
            for x_Data in self.x_data:
                print(x_Data)
            for y_Data in self.y_data:
                print(y_Data)
            for speed_Data in self.speed:
                print(speed_Data)
    
    def ClickMethod(self):

            for j in range(100):
                x,y=pyautogui.position()
                print(x,y)
                time.sleep(0.001)
            # 안쪽 리스트로 사용할 빈 리스트 생성
            line_x = []
            line_y = []
            line_speed=[]

            for j in range(100):
                x,y=pyautogui.position()
                line_x.append(x)
                line_y.append(y)# 안쪽 리스트에 0 추가
                time.sleep(0.001)

            self.x_data.append(line_x)
            self.y_data.append(line_y)
            
            for i in range(1, len(line_x)):
                Speed=((line_x[i]-line_x[i-1])**2) + ((line_y[i]-line_y[i-1])**2)
                Speed=math.sqrt(Speed)
                line_speed.append(Speed)
            self.speed.append(line_speed)
            
    def Start(self):
         
            # 안쪽 리스트로 사용할 빈 리스트 생성
            line_x = []
            line_y = []
            line_speed=[]

            for j in range(100): #1초동안 데이터 수집 
                x,y=pyautogui.position()
                line_x.append(x)
                line_y.append(y)# 안쪽 리스트에 0 추가
                time.sleep(0.001)

            self.x_data.append(line_x)
            self.y_data.append(line_y)
            
            for i in range(len(line_x)):
                Speed=((line_x[i]-line_x[i-1])**2) + ((line_y[i]-line_y[i-1])**2)
                Speed=math.sqrt(Speed)
                line_speed.append(Speed)
            self.speed.append(line_speed)
            
            testSet=[]
            
            for i in range(1,len(line_x)):
            
                test=[] #리스트 하나당 임시로 x,y,속력 저장 리스트
                test.append(line_x[i])
                test.append(line_y[i])
                test.append(line_speed[i])
                testSet.append(test)
            
            Test=[]
            Test.append(testSet)
            
            Test_set = np.array(Test)
            
            print(Test_set)
            
            testPredict = model.predict(Test_set)
        
           # print("TestSet 값 출력 : ", TestSet)
            
            x_position=testPredict[0,98,0] #TestSet[0,98,0] #x값
            y_position=testPredict[0,98,1] #TestSet[0,98,1] #y값
            
            print("x y 값 출력 %d %d ", x_position, y_position)
            n=0  #결과에따라 사이트 이동
            while(n==0):
                if x_position>400 and x_position<950 :
                    if 320<y_position and y_position<480:
                        #kwu
                        url = 'https://www.kw.ac.kr/ko/'
                        webbrowser.open(url)
                    if 480<y_position and y_position<800:
                        #naver
                        url = 'https://www.naver.com/'
                        webbrowser.open(url)
                if 950<x_position and x_position<1100:
                    if 320<y_position and y_position<480:
                        #google
                        url = 'https://www.google.co.kr/'
                        webbrowser.open(url)
                    if 480<y_position and y_position<800:
                        #youtube
                        url = 'https://www.youtube.com/'
                        webbrowser.open(url)
                n+=1;
                              

app = QApplication([])
ex = WindowClass()
sys.exit(app.exec_())

864 381
864 381
867 381
867 381
870 381
870 381
875 381
875 381
882 381
882 381
887 380
887 380
893 378
893 378
900 377
907 377
907 377
914 377
914 377
920 377
920 377
926 377
926 377
932 377
932 377
936 377
936 377
940 377
940 377
944 377
944 377
947 377
947 377
950 377
952 378
953 378
953 378
953 378
953 378
953 378
953 378
953 380
953 380
953 380
953 380
953 382
953 382
953 382
953 382
953 382
953 382
953 382
953 382
953 382
953 382
953 382
953 382
953 382
953 382
953 382
953 382
953 382
953 382
953 382
953 382
953 382
953 383
953 383
953 383
953 383
953 383
953 383
953 383
953 383
954 384
954 384
955 385
955 385
956 385
956 385
957 385
957 385
957 385
957 385
961 388
961 388
961 388
961 388
961 388
961 388
961 388
961 388
961 388
961 388
961 388
961 388
961 388
961 388
961 388
961 388
918 421
912 426
912 426
906 430
906 430
902 433
902 433
898 436
898 436
895 438
895 438
892 440
892 440
889 442
889 442
887 443
887 443
887 446
887 446
885 448
885 448
883 449
883 449
882 452
882 452


SystemExit: 0

C:\Users\meid9\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
